In [1]:
#Dataframe, Image processing
import os
import pandas as pd
from PIL import Image

#Deepleaning: PYTORCH
import os
from torchvision import datasets, transforms, models
from torch.utils.data import DataLoader
import torch
import torch.nn as nn
import torch.optim as optim
from PIL import ImageFile
ImageFile.LOAD_TRUNCATED_IMAGES = True

#EXPLAINABLE AI: TORCHVISION AND GRAD CAM
from torchvision import transforms
from PIL import Image
import torch
from pytorch_grad_cam import GradCAM
from pytorch_grad_cam.utils.image import show_cam_on_image

#UI: WIDGETS
import ipywidgets as widgets
from IPython.display import display, clear_output
from PIL import Image
import io
# Display the result
import matplotlib.pyplot as plt

<h2>DATA EXPLORATION</h2>

In [19]:
main_dir = '<ENTER_PARENT_DIRECTORY_NAME'
dir_train_in_wildfire = main_dir+'/train/wildfire'
dir_train_in_nowildfire = main_dir+'./train/nowildfire'

In [3]:
# List all files in the directory

def get_location(directory_path):
    lst_items_out = []
    lst_header_out = ['longitude','latitude']
    try:
        filenames = os.listdir(directory_path)
        #print("Files and directories in '", directory_path, "':")
        for filename in filenames:
            fname = filename.replace('.jpg','')
            fname = fname.split(',')
            longitude = fname[0]
            latitude = fname[1]
            lst_items_out.append([longitude,latitude])
            #print(filename)
    except FileNotFoundError:
        print("The directory does not exist.")
    except PermissionError:
        print("Permission denied.")

    return lst_items_out, lst_header_out




In [4]:
lst_items_train, lst_header_train = get_location(dir_train_in_wildfire)

In [5]:
df = pd.DataFrame(lst_items_train)
df.columns = ['longitide','latitude']
df.head(5)

,longitide,latitude
0,-75.79731,47.6256
1,-69.5572,51.9064
2,-79.11752,47.38459
3,-72.743,45.7617
4,-75.7764,45.4516


In [7]:
file_out = main_dir+'/wildfire_locations.csv'
df.to_csv(file_out, index=False)

<h2>Get geospatial visualisation with datajarvis.ai</h2>

In [8]:
#Upload CSV at datajarvis.ai
#Prompt - give a geospatial heatmap with Longitide as Longitide and Latitude as Latitude

<h2>MODEL PYTORCH</h2>

In [10]:
# Set device
#device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device = torch.device("mps" if torch.backends.mps.is_available() else "cpu")
print ('device..',device)
# Paths
data_dir = main_dir+"/train"  # Replace with your root data directory

# Transformations
transform = transforms.Compose([
    transforms.Resize((128, 128)),  # Resize all images to 128x128
    transforms.ToTensor(),  # Convert images to tensors
    transforms.Normalize([0.5, 0.5, 0.5], [0.5, 0.5, 0.5])  # Normalize images
])

# Load datasets
train_data = datasets.ImageFolder(root=data_dir, transform=transform)
train_loader = DataLoader(train_data, batch_size=32, shuffle=True)

# Define the model (simple CNN)
class CNNModel(nn.Module):
    def __init__(self):
        super(CNNModel, self).__init__()
        self.conv_layers = nn.Sequential(
            nn.Conv2d(3, 32, kernel_size=3, stride=1, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2),
            
            nn.Conv2d(32, 64, kernel_size=3, stride=1, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2)
        )
        self.fc_layers = nn.Sequential(
            nn.Flatten(),
            nn.Linear(64 * 32 * 32, 128),
            nn.ReLU(),
            nn.Dropout(0.5),
            nn.Linear(128, 2)  # Output: 2 classes (wildfire, no wildfire)
        )

    def forward(self, x):
        x = self.conv_layers(x)
        x = self.fc_layers(x)
        return x

model = CNNModel().to(device)

device.. mps


In [11]:
# Loss and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Training loop
num_epochs = 10
for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0

    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)

        # Forward pass
        outputs = model(images)
        loss = criterion(outputs, labels)

        # Backward pass and optimization
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        running_loss += loss.item()

    print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {running_loss/len(train_loader):.4f}")

print("Training complete!")


Epoch [1/10], Loss: 0.2057
Epoch [2/10], Loss: 0.1498
Epoch [3/10], Loss: 0.1269
Epoch [4/10], Loss: 0.1077
Epoch [5/10], Loss: 0.0878
Epoch [6/10], Loss: 0.0684
Epoch [7/10], Loss: 0.0543
Epoch [8/10], Loss: 0.0511
Epoch [9/10], Loss: 0.0407
Epoch [10/10], Loss: 0.0384
Training complete!


In [13]:
# Save the model
wildfire_model_path = main_dir+'/wildfire_model.pth'
torch.save(model.state_dict(), wildfire_model_path)
#print("Model saved as "+wildfire_model_path)

In [14]:
#Predict
# Initialize the model
device = torch.device("mps" if torch.backends.mps.is_available() else "cpu")
model = CNNModel().to(device)

# Load model weights
model.load_state_dict(torch.load(wildfire_model_path, map_location=device))
model.eval()

# Define preprocessing transformations
transform = transforms.Compose([
    transforms.Resize((128, 128)),  # Resize to match training size
    transforms.ToTensor(),
    transforms.Normalize([0.5, 0.5, 0.5], [0.5, 0.5, 0.5])  # Same normalization as training
])


/var/folders/vr/mpmvnq3921n9j9rgk4zmcltw0000gp/T/ipykernel_82501/4270485815.py:7: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load(wildfire_mod

In [15]:
#Predict
# Function to predict wildfire or no wildfire
def predict_image(image_path):
    image = Image.open(image_path).convert('RGB')  # Load image
    image = transform(image).unsqueeze(0).to(device)  # Apply transformations and add batch dimension
    
    # Get predictions
    with torch.no_grad():
        outputs = model(image)
        _, predicted = torch.max(outputs, 1)  # Get class index
    
    # Map class index to label
    classes = {0: "NO WILDFIRE RISK", 1: "WILDFIRE RISK"}
    return classes[predicted.item()]


In [16]:
# Predict images from a folder

def read_image_and_predict(folder_path):
    for file_name in os.listdir(folder_path):
        file_path = os.path.join(folder_path, file_name)
        if os.path.isfile(file_path):  # Ensure it's a file
            prediction = predict_image(file_path)
            #print(f"Image: {file_name} => Prediction: {prediction}")

In [13]:
wildfire_path = main_dir+"/wildfire/valid/wildfire"  # Replace with the folder containing test images
#read_image_and_predict(wildfire_path)

In [14]:
wildfire_path = main_dir+"/valid/nowildfire"  # Replace with the folder containing test images
#read_image_and_predict(wildfire_path)

<h2>Visualizations</h2>

In [17]:


def get_prediction_and_explaination(image_path):
    # Preprocessing transforms (adjust based on your model's requirements)
    transform = transforms.Compose([
        transforms.Resize((128, 128)),  # Resize to the input size of your model
        transforms.ToTensor(),          # Convert to tensor
        transforms.Normalize([0.5, 0.5, 0.5], [0.5, 0.5, 0.5])  # Normalize (same as during training)
    ])

    # Load and preprocess the input image
#image_path = "./valid/wildfire/-57.25,51.51.jpg"  # Replace with your image path

    prediction = predict_image(image_path)
    print(f"Image: {image_path} => Prediction: {prediction}")

    image = Image.open(image_path).convert('RGB')  # Open the image and ensure it's in RGB
    input_tensor = transform(image).unsqueeze(0)  # Add batch dimension
    input_tensor = input_tensor.to(device)  # Move to the same device as the model




    # Target layer for Grad-CAM (e.g., the first convolutional layer)
    target_layer = model.conv_layers[0]  # Adjust based on your model structure

    # Initialize Grad-CAM
    cam = GradCAM(model=model, target_layers=[target_layer])

    # Generate the Grad-CAM heatmap
    grayscale_cam = cam(input_tensor=input_tensor)[0]  # [0] extracts the heatmap for the first image

    # Convert the original image to a format compatible with `show_cam_on_image`
    original_image = input_tensor.squeeze(0).permute(1, 2, 0).cpu().numpy()  # Convert to HWC format
    original_image = (original_image - original_image.min()) / (original_image.max() - original_image.min())  # Normalize for display

    # Overlay the heatmap on the original image
    heatmap = show_cam_on_image(original_image, grayscale_cam, use_rgb=True)

    return prediction,heatmap


<b>UI</b>

In [18]:


# Directory to save uploaded images
UPLOAD_DIR = "uploaded_images"
os.makedirs(UPLOAD_DIR, exist_ok=True)

# Function to handle image selection
def on_image_upload(change):
    # Get the uploaded image
    uploaded_file = change['new']
    if uploaded_file:
        content = uploaded_file[0]['content']
        img = Image.open(io.BytesIO(content))
        img.thumbnail((300, 300))  # Resize for display
        with output:
            clear_output(wait=True)
            display(img)

# Function to handle prediction
def predict_image1(_):
    
    if image_upload.value:
        # Simulate a prediction function
        fname = image_upload.value[0]['name']
        img_path = main_dir+'/predict/'+fname
        print ('img_path..',img_path)
        prediction,heatmap = get_prediction_and_explaination(img_path)
        
        result = "Prediction: "+prediction
        with output:
            display("Prediction complete!")
            display(result)

          

            plt.imshow(heatmap)
            plt.axis('off')
            plt.show()
    else:
        with output:
            display("Please upload an image first.")

# Widgets
image_upload = widgets.FileUpload(accept='image/*', multiple=False, description="Upload Image")
print (image_upload)
image_upload.observe(on_image_upload, names='value')

predict_button = widgets.Button(description="Predict")
predict_button.on_click(predict_image1)

output = widgets.Output()

# Display the UI
ui = widgets.VBox([image_upload, predict_button, output])
display(ui)

FileUpload(value=(), accept='image/*', description='Upload Image')
